In [1]:
# Initiate a Language Model

import re
import string
import math
import numpy as np
import random

# Functions to: "read" text file to be encoded.
# Lower case, remove non-letters.  Return tokens. 
# break corpus into tokens, calculate probability of N-grams.

def CalculateModel(fname):
    # initial state distribution
    pi = np.zeros(26)
        
    # initialize Markov matrix
    M = np.ones((26, 26))

    regex = re.compile('[^a-zA-Z]')

    for line in open(fname, 'r', encoding='utf8'):
        line = line.rstrip()
        line = regex.sub(' ', line)
        line = line.lower()
        tokens = line.split()

        for token in tokens:
            ch0 = token[0]
            i = ord(ch0) - 97
            pi[i] += 1

            for ch1 in token[1:]:
                i = ord(ch0) - 97
                j = ord(ch1) - 97
                M[i,j] += 1
                ch0 = ch1
            
    pi /= pi.sum()
    M /= M.sum(axis=1, keepdims=True)
    
    return pi, M

# Initiate the language model

# initial state distribution
LM_pi = np.zeros(26)

# initialize Markov matrix
LM_M = np.ones((26, 26))

# File to base language model on; this one used Moby Dick
fname = 'C:\DataScience\machine_learning_examples\large_files\mb.txt'
LM_pi, LM_M = CalculateModel(fname)

In [2]:
#  Function to Encode it!
#  Read each letter.  If it's a lower case letter, return the cipher letter, which is capitalizezd.
#  Return spaces and puctuation as they are in plaintext.
#  For each letter in plaintext, look for it in the Key, if found, replace, if not, return same character in CIPHER

def Encode(plaintext):
    Letters, Key = MakeKey()

    CIPHER = ""

    for x in plaintext:
        Found = False
        n = 0
        for y in Key:
            if x == Letters[n]:
                CIPHER = CIPHER + Key[n]
                Found = True
                break
            n+=1
        if not Found:
            CIPHER = CIPHER + x
    return CIPHER, Key

#  Function to create random substitution cipher

def MakeKey():
    Letters = ["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"]
    Key = Letters.copy()
    random.shuffle(Key)
    #Simple shift for testing
    #Key = ["b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z","a"]
    n = 0
    for x in Key:
        Key[n] = x.upper()
        n+=1
    return Letters, Key

In [7]:
# Decode using the language model, probabilities, and genetic algorithm

# Functions to compute string probability based on an input

def WordProb(word):
    i = ord(word[0]) - 97
    logp = np.log(LM_pi[i])

    for ch in word[1:]:
        j = ord(ch) - 97
        logp += np.log(LM_M[i, j]) # update prob
        i = j # update j

    return logp

def StringProb(words):
    if type(words) == str:
        words = words.split()

    logp = 0
    for word in words:
        logp += WordProb(word)
    return logp

#  Function to Decode it!
#  Read each letter.  If it's in the key, return the plain letter, which is lowercased.
#  Return spaces and puctuation as spaces.

def Decode(CIPHER, Key, Letters):

    plain =""

    for x in CIPHER:
        Found = False
        n = 0
        for y in Key:
            if x == Key[n]:
                plain = plain + Letters[n]
                Found = True
                break
            n+=1
        if not Found:
            plain = plain + " "
    return plain

# Function to swap two characters, letters in a key

def MutateKey(Key, num_mutations):
    idx = range(len(Key))
    for x in range(num_mutations):
        i1, i2 = random.sample(idx, 2)
        MutatedKey = Key.copy()
        MutatedKey[i1], MutatedKey[i2] = MutatedKey[i2], MutatedKey[i1]
    return MutatedKey

# Genetic algorithm; generate num_tests random keys; nested for loop
# for loop of epochs
# 1st epoch: create matrix of num_tests of letters, keys -> Results
# 1 - Test all num_tests -> Scores
# 2 - Find highest X Scores
# 3 - Create num_children of top_x -> Results
# 4 - Create matrix of top_x + children
# 5 - Repeat

def GenaticAlgorithm(MYSTERY):

    TestKey = []
    TestLetters = []

    epochs = 100000
    num_tests = 20
    top_x = 1
    num_children = (num_tests - top_x)//top_x  # NEED TO FIX SO NO OUT OF RANGES
    num_mutations = 1

    # Create the initial test keys, letters
    for y in range(num_tests):    
        Test_Letters, Test_Key = MakeKey()
        TestLetters.append(Test_Letters)
        TestKey.append(Test_Key)

    for x in range(epochs):      

        NewTestKey = []
        Scores = []

        i = 0
        for y in TestKey:
            Test_BiKey = []
            Test_Probability = 0.0
            Test = Decode(MYSTERY,TestKey[i],TestLetters[i])
            Scores.append(StringProb(Test))
            i+=1

        # Get the top 5     
        top_x_idx = np.argsort(Scores)[-top_x:]
        top_x_values = [Scores[i] for i in top_x_idx]

        # Remember the "best" for later
        Best_idx = int(np.argsort(top_x_idx)[-1:])
        BestKey = TestKey[Best_idx]
        BestLetters = TestLetters[Best_idx]
        Best = Decode(MYSTERY,BestKey,BestLetters)

        for z in top_x_idx:
            #Put top X in new arrays
            #Make num_children mutations from each top X
            NewTestKey.append(TestKey[z])
            for y in range(num_children):
                Child_Key = MutateKey(TestKey[z], num_mutations)
                NewTestKey.append(Child_Key)

        TestKey = NewTestKey

    # Print the "best" decode!

    Best = Decode(MYSTERY,BestKey,BestLetters)
    return Best

In [8]:
# Let's make a cipher, or add one of our own, for deciphering!

def MakeCipher():
    # plaintext = open('C:\DataScience\machine_learning_examples\large_files\sh.txt','r', encoding='utf8').read()
    # OR USE 
    plaintext = '''Human experience shows that people, not organizations or management systems, 
    get things done. For this reason, subordinates must be given authority and responsibility early in their 
    careers. In this way they develop quickly and can help the manager do his work. The manager, of course, 
    remains ultimately responsible and must accept the blame if subordinates make mistakes.
    As subordinates develop, work should be constantly added so that no one can finish his job. 
    This serves as a prod and a challenge. It brings out their capabilities and frees the manager to assume 
    added responsibilities. As members of the organization become capable of assuming new and more 
    difficult duties, they develop pride in doing the job well. This attitude soon permeates the 
    entire organization.'''
    plaintext = plaintext.lower()
    CIPHER, CIPHER_KEY = Encode(plaintext)
    return CIPHER, CIPHER_KEY

#MYSTERY, MYSTERY_KEY = MakeCipher()

# OR HAVE YOUR OWN CIPHER YOU FOUND SOMEWHERE AND WANT TO DECIPHER

#MYSTERY = '''BT JPX RMLX PCUV AMLX ICVJP IBTWXVR CI M LMT’R PMTN, MTN YVCJX CDXV MWMBTRJ 
#JPX AMTNGXRJBAH UQCT JPX QGMRJXV CI JPX YMGG CI JPX HBTW’R QMGMAX; MTN JPX HBTW RMY JPX QMVJ 
#CI JPX PMTN JPMJ YVCJX. JPXT JPX HBTW’R ACUTJXTMTAX YMR APMTWXN, MTN PBR JPCUWPJR JVCUFGXN PBL, 
#RC JPMJ JPX SCBTJR CI PBR GCBTR YXVX GCCRXN, MTN PBR HTXXR RLCJX CTX MWMBTRJ MTCJPXV. JPX HBTW 
#AVBXN MGCUN JC FVBTW BT JPX MRJVCGCWXVR, JPX APMGNXMTR, MTN JPX RCCJPRMEXVR. MTN JPX HBTW RQMHX, 
#MTN RMBN JC JPX YBRX LXT CI FMFEGCT, YPCRCXDXV RPMGG VXMN JPBR YVBJBTW, MTN RPCY LX JPX 
#BTJXVQVXJMJBCT JPXVXCI, RPMGG FX AGCJPXN YBJP RAM'''

MYSTERY = '''KDRNHK TXUC RKLL YX.'''

print(MYSTERY)
decipher = GenaticAlgorithm(MYSTERY)

print("The decode is:\n",decipher)

KDRNHK TXUC RKLL YX.
The decode is:
 ayonga ster oall it 
